In [1]:
! pip install langchain-neo4j

In [1]:
from langchain_neo4j import Neo4jGraph

In [3]:
Neo4jGraph(url='bolt://localhost:7687',username='neo4j',password='password123',database='mydatabase')

In [6]:
!pip install -U langchain langchain-community

  Using cached langchain-0.3.15-py3-none-any.whl.metadata (7.1 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 1.1 MB/s eta 0:00:00 MB/s eta 0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 1.1 MB/s eta 0:00:001.1 MB/s eta 0:00:01
Using cached httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.29
    Uninstalling langchain-core-0.3.29:
      Successfully uninstalled langchain-core-0.3.29
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.14
    Uninstalling langchain-0.3.14:
      Successfully uninstalled langchain-0.3.14
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.3.3
    Uninstalling langchain-community-0.3.3:
      Successfully uninstalled langchain-community-0.3.3


In [ ]:
import os
from typing import Optional, List
from dotenv import load_dotenv

from pydantic import Field
from langchain_community.utilities import GoogleSerperAPIWrapper
import langchain
from langchain_core.output_parsers import JsonOutputParser
from dotenv import load_dotenv
import pprint
load_dotenv()
from pydantic import BaseModel, Field
from langchain_core.runnables.base import RunnableLambda


class KnowledgeGraph(BaseModel):
    type: str
    description: str
    description_link: str = Field(..., alias='descriptionLink')
    attributes: dict

class Sitelink(BaseModel):
    title: str
    link: str

class OrganicItem(BaseModel):
    title: str
    link: str
    snippet: str
    sitelinks: Optional[list[Sitelink]] = None
    position: int
    date: Optional[str] = None

class TopStory(BaseModel):
    title: str
    link: str
    source: str
    date: str

class SearchResults(BaseModel):
    organic: List[OrganicItem]

from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(template="""filetype:pdf base knowledge documents on {input}""")



def serperfetch(query:str):
    search_results=GoogleSerperAPIWrapper(type='search').results(query)
    return SearchResults.model_validate({'organic':search_results['organic']})

serper_chain=prompt | RunnableLambda(lambda x: serperfetch(x))
serperfetch("filetype:pdf base knowledge documents on Routing Guides by Lane ")

In [3]:
serperfetch("filetype:pdf base knowledge documents on Routing Guides by Lane ")

ValidationError: 1 validation error for GoogleSerperAPIWrapper
  Value error, Did not find serper_api_key, please add an environment variable `SERPER_API_KEY` which contains it, or pass `serper_api_key` as a named parameter. [type=value_error, input_value={'type': 'search'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error